In [47]:
from dotenv import load_dotenv
import os
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.core import SimpleDirectoryReader
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex

In [48]:
load_dotenv()

True

In [49]:
api_key = os.environ["OPENAI_API_KEY"]

In [50]:
Settings.llm = OpenAI(model = "gpt-4o-mini")

In [51]:
def load_documents(directory):
    documents = SimpleDirectoryReader(directory).load_data()
    return documents

In [52]:
def create_index(db, documents, collection_name):
    chroma_collection = db.get_or_create_collection(collection_name)
    
    vector_store = ChromaVectorStore(chroma_collection = chroma_collection)
    storage_context = StorageContext.from_defaults(vector_store = vector_store)
    
    index = VectorStoreIndex.from_documents(
        documents, storage_context = storage_context
    )
    return index



In [54]:
docs = load_documents("./data")
db = chromadb.PersistentClient(path = "./db")
index = create_index(db, docs, "custom_files")


In [55]:
query_engine = index.as_query_engine()
response = query_engine.query("What does this document contain?")
print(response)

This document contains the consolidated balance sheets of Alphabet Inc. as of December 31, 2023, and September 30, 2024. It details the company's assets, liabilities, and stockholders' equity, including current and non-current assets, current and long-term liabilities, and the breakdown of stockholders' equity components.


In [56]:
query_engine = index.as_query_engine()
response = query_engine.query("What was Alphabet's revenue in Q3 2024?")
print(response)


Alphabet's revenue in Q3 2024 was $88.3 billion.


In [57]:
response = query_engine.query("How much did Alphabet's revenue increase yoy in Q3 2024?")
print(response)

Alphabet's revenue increased by 15% year over year in Q3 2024.


In [58]:
from llama_index.core.evaluation import FaithfulnessEvaluator
from llama_index.core.evaluation import RelevancyEvaluator

In [59]:
llm = OpenAI(model="gpt-4o-mini", temperature=0.0)

In [60]:
import nest_asyncio
nest_asyncio.apply()

faithfulness_evaluator = FaithfulnessEvaluator(llm = llm)
eval_result = faithfulness_evaluator.evaluate_response(response=response)
print(eval_result.score)
print(eval_result.passing)

nest_asyncio.apply()
relevancy_evaluator = RelevancyEvaluator(llm=llm)
eval_result = relevancy_evaluator.evaluate_response(query = "How much did Alphabet's revenue increase yoy in Q3 2024?", response=response)
print(eval_result.score)
print(eval_result.passing)



1.0
True
1.0
True


In [61]:
def evaluate_faithfulness(query, response) -> tuple[float, bool]:
    faithfulness_evaluator = FaithfulnessEvaluator(llm = llm)
    eval_result = faithfulness_evaluator.evaluate_response(query = query, response=response)
    return (eval_result.score, eval_result.passing)

evaluate_faithfulness("How much did Alphabet's revenue increase yoy in Q3 2024?", response)

(1.0, True)

In [62]:
def evaluate_relevancy(query, response) -> tuple[float, bool]:
    relevancy_evaluator = RelevancyEvaluator(llm=llm)
    eval_result = relevancy_evaluator.evaluate_response(query = query, response=response)
    return (eval_result.score, eval_result.passing)

evaluate_relevancy("How much did Alphabet's revenue increase yoy in Q3 2024?", response)

(1.0, True)